In [5]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils import class_weight

# Define the dataset path and other parameters
dataset_path = '/Users/mohammadadnaan/Downloads/7thSem/MajorProject/Model/tinyDataSet/ucid'
image_size = (64, 64)  # Image size used in the paper
batch_size = 32

# Data augmentation and rescaling
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load training images from directories
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Load validation images from directories
validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Calculate class weights to address class imbalance
class_labels = train_generator.classes

# Correct usage of compute_class_weight
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(class_labels),
    y=class_labels
)

# Convert the result into a dictionary to use in model.fit
class_weights = dict(enumerate(class_weights))

# Increase model complexity
model = models.Sequential()

# Input layer
model.add(layers.Input(shape=(64, 64, 3)))

# First set of Conv + BatchNorm + Pool layers
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

# Second set of Conv + BatchNorm + Pool layers
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

# Third set of Conv + BatchNorm + Pool layers
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

# Fully connected layers
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))

# Output layer
model.add(layers.Dense(train_generator.num_classes, activation='softmax'))

# Compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-4),  # Reduced learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)


Found 8242 images belonging to 10 classes.
Found 2060 images belonging to 10 classes.


In [7]:
# Train the model with class weights and learning rate scheduler
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator,
    class_weight=class_weights,  # Handle class imbalance
    callbacks=[early_stopping, lr_scheduler]
)


Epoch 1/100


/Users/mohammadadnaan/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


258/258 ━━━━━━━━━━━━━━━━━━━━ 78s 250ms/step - accuracy: 0.0982 - loss: 3.3237 - val_accuracy: 0.0990 - val_loss: 3.1396 - learning_rate: 1.0000e-04
Epoch 2/100
258/258 ━━━━━━━━━━━━━━━━━━━━ 69s 262ms/step - accuracy: 0.1054 - loss: 2.4513 - val_accuracy: 0.1083 - val_loss: 2.3673 - learning_rate: 1.0000e-04
Epoch 3/100
258/258 ━━━━━━━━━━━━━━━━━━━━ 61s 234ms/step - accuracy: 0.1027 - loss: 2.3899 - val_accuracy: 0.1073 - val_loss: 2.3082 - learning_rate: 1.0000e-04
Epoch 4/100
258/258 ━━━━━━━━━━━━━━━━━━━━ 65s 247ms/step - accuracy: 0.0975 - loss: 2.3690 - val_accuracy: 0.1097 - val_loss: 2.3037 - learning_rate: 1.0000e-04
Epoch 5/100
258/258 ━━━━━━━━━━━━━━━━━━━━ 61s 232ms/step - accuracy: 0.1034 - loss: 2.3495 - val_accuracy: 0.1005 - val_loss: 2.3032 - learning_rate: 1.0000e-04
Epoch 6/100
258/258 ━━━━━━━━━━━━━━━━━━━━ 60s 228ms/step - accuracy: 0.0945 - loss: 2.3407 - val_accuracy: 0.1000 - val_loss: 2.3028 - learning_rate: 1.0000e-04
Epoch 7/100
258/258 ━━━━━━━━━━━━━━━━━━━━ 88s 252ms/s

In [ ]:
# Save the trained model
model.save('ucid_cnn_model_with_improvements.h5')

# Evaluate the model on validation data
val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_acc}")